<a href="https://colab.research.google.com/github/AdarshShah/final-year-project/blob/master/project_sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [0]:
import pandas as pd
import psycopg2
import getpass
import numpy as np
import tensorflow as tf

# Declare Global Variables

In [0]:
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'
password = 'postgres'
icd9_code = '99592'

# Create Database Connection

In [0]:
conn = psycopg2.connect(user=user,password=password,dbname=dbname,host=host)
cur = conn.cursor()
cur.execute('SET search_path to {}'.format(schema))

# Load Patient data
> Data from admissions suffering from severe sepsis



In [0]:
query = """SELECT DISTINCT * FROM mimiciii.admissions i 
INNER JOIN mimiciii.diagnoses_icd USING(hadm_id)
INNER JOIN mimiciii.d_icd_diagnoses USING(icd9_code)
WHERE icd9_code LIKE '99592';
"""
admissions = pd.read_sql_query(sql=query,con=conn)

> List Items from d_labitems and d_items

In [0]:
#d_items = ['Platelets','C Reactive Protein (CRP)','pH (Art)','HCO3','TCO2 (cap)']
#d_labitems = ['Leukocytes','Urea Nitrogen','Creatinine','Glucose','Calcium, Total','Sodium','Potassium','Bilirubin','Albumin','Lactate','pCO2','pO2','Hemoglobin']

#Append itemid below !!!!
d_item_ce = ['0','828','3789','6256','791','3750','1525','220615','811','3744','3745','1529','226537','837','3803','1536','4948','3066','227444']  
d_item_cv = ['0','30006','44711','44441']
d_item_mv = ['0','225170']
d_labitem = ['51486','51006','50912','50809','50931','51478','50813','50820']

In [0]:
def get_pandas(items, table):
  query = "SELECT DISTINCT * FROM mimiciii."+table+" WHERE itemid IN " + str(tuple(items))
  items = pd.read_sql_query(sql=query,con=conn)
  return items

d_item_ce = get_pandas(d_item_ce,"d_items")
d_item_cv = get_pandas(d_item_cv,"d_items")
d_item_mv = get_pandas(d_item_mv,"d_items")
d_labitem = get_pandas(d_labitem,"d_labitems")

# Fetch items from chartevents

In [0]:
query = "SELECT * FROM mimiciii.chartevents WHERE itemid IN " + str(tuple(d_item_ce['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
chartevents = pd.read_sql_query(sql=query,con=conn)


In [0]:
chartevents.describe()

# Fetch items from inputevents_cv



In [0]:
query = "SELECT * FROM mimiciii.inputevents_cv WHERE itemid IN " + str(tuple(d_item_cv['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
inputevents_cv = pd.read_sql_query(sql=query,con=conn)

In [0]:
inputevents_cv.describe()

# Fetch items from inputevents_mv

In [0]:
query = "SELECT * FROM mimiciii.inputevents_mv WHERE itemid IN ('225170') AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,starttime,itemid;"
inputevents_mv = pd.read_sql_query(sql=query,con=conn)

In [0]:
inputevents_mv.describe()

# Fetch items from labevents

In [0]:
query = "SELECT * FROM mimiciii.labevents WHERE itemid IN " + str(tuple(d_labitem['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
labevents = pd.read_sql_query(sql=query,con=conn)

In [0]:
labevents.describe()

# Generate Tensors of chartevents

In [0]:
hadm_id = chartevents['hadm_id'].drop_duplicates()

In [0]:
dataset_ce = list()
for hadm in hadm_id:
  temp = chartevents[chartevents['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='valuenum')
  dataset_ce.append(temp)

# Generate Tensors of inputevents_cv

In [0]:
hadm_id = inputevents_cv['hadm_id'].drop_duplicates()

In [0]:
dataset_cv = list()
for hadm in hadm_id:
  temp = inputevents_cv[inputevents_cv['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='amount')
  dataset_cv.append(temp)

# Generate Tensors of inputevents_mv

In [0]:
hadm_id = inputevents_mv['hadm_id'].drop_duplicates()

In [0]:
dataset_mv = list()
for hadm in hadm_id:
  temp = inputevents_mv[inputevents_mv['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='amount')
  dataset_mv.append(temp)

# Generate Tensors of labevents

In [0]:
hadm_id = labevents['hadm_id'].drop_duplicates()

In [0]:
dataset_lab = list()
for hadm in hadm_id:
  temp = labevents[labevents['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='valuenum')
  dataset_lab.append(temp)